# DNA Center SDK

In [1]:
from dnac_config import DNAC, DNAC_PORT
import json

# this is only requiired for proxy change
import requests
requests.packages.urllib3.disable_warnings()
print(DNAC)

sandboxdnac2.cisco.com


In [2]:
from dnacentersdk import api

In [3]:
# only need to verify=False as going to change thte base_url later due to proxy
dnac = api.DNACenterAPI(username="devnetuser",
                        password="Cisco123!",
                        base_url="https://{}:443".format(DNAC),verify=False)
print(dnac.base_url)

https://sandboxdnac2.cisco.com:443


In [4]:
devices = dnac.devices.get_device_list()

#### By default the object that is returned can be rendered as a json.dict

In [5]:
print(json.dumps(devices,indent=2))

{
  "response": [
    {
      "location": null,
      "type": "Cisco 3504 Wireless LAN Controller",
      "errorCode": "ERROR-CONNECTION-CLOSED",
      "family": "Wireless Controller",
      "role": "ACCESS",
      "lastUpdateTime": 1566708369489,
      "serialNumber": "FCW2218M0B1",
      "collectionInterval": "Global Default",
      "inventoryStatusDetail": "<status><general code=\"FAILED_FEAT\"/><failed_features names=\"com.cisco.apicem.feature.wlc.feature.wlcFabricProperties,com.cisco.apicem.feature.avc_wlc_fabric_feature,com.cisco.apicem.feature.Mobilitytunnel,com.cisco.apicem.feature.apicem_memoryfeature_wlc,com.cisco.apicem.feature.neinfo_capwaptunnel_wlc_feature,com.cisco.apicem.feature.feature_avc_wlc,com.cisco.enc.apic.feature.host_feature_wlc,com.cisco.apicem.feature.haStatus,com.cisco.apicem.feature.wlc.flexAclTemp,com.cisco.ifm.feature.LAGInfo\" code=\"ERROR_CONNECTION_CLOSED\"/><topCause code=\"ERROR_CONNECTION_CLOSED\"/>\n</status>",
      "softwareType": "Cisco Controll

In [8]:
# old way.. json attributes
data = ", ".join([ "{}:{}".format(dev['managementIpAddress'],dev['platformId'])
                  for dev in devices['response']])

#### Example of an object

In [9]:
# new way... objects  dev.managementIpAddress vs dev['managementIpAddress']
data = ", ".join([ "{}:{}".format(dev.managementIpAddress,dev.platformId)
                  for dev in devices.response])

In [10]:
print(data)

10.10.20.51:AIR-CT3504-K9, 10.10.20.81:C9300-48U, 10.10.20.82:C9300-48U, 10.10.20.80:WS-C3850-24P-L, 10.10.20.241:AIR-AP1141N-A-K9, 10.10.20.250:AIR-AP1141N-A-K9, 10.10.20.242:AIR-AP1141N-A-K9, 10.10.20.243:AIR-AP1141N-A-K9, 10.10.20.244:AIR-AP1141N-A-K9, 10.10.20.245:AIR-AP1141N-A-K9, 10.10.20.246:AIR-AP1141N-A-K9, 10.10.20.247:AIR-AP1141N-A-K9, 10.10.20.248:AIR-AP1141N-A-K9, 10.10.20.249:AIR-AP1141N-A-K9


## Templates

In [11]:
templates = dnac.template_programmer.gets_the_templates_available()

In [12]:
template = [ t for t in templates if t.projectName == "switching" and t.name == "interface-des"][0]

In [13]:
print(json.dumps(template, indent=2))

{
  "name": "interface-des",
  "projectName": "switching",
  "projectId": "dda1d3b7-788e-4608-8994-ab9ab676b03b",
  "templateId": "8a91e0f1-de37-409b-8242-b344825833df",
  "versionsInfo": [
    {
      "id": "bd8fd41f-e449-441f-9ac1-804686a87c55",
      "description": "",
      "author": "admin",
      "version": "1",
      "versionComment": "",
      "versionTime": 1547865199368
    }
  ],
  "composite": false
}


In [14]:
# this is the ID of the template object.
templateId= template['templateId']


In [15]:
print(templateId)

8a91e0f1-de37-409b-8242-b344825833df


In [16]:
# this will find the latest version of the template (inside the object).  NOTE The ID are different..
latest_id = 0
max = 0
for v in template['versionsInfo']:
    if int(v['version']) > max:
        max = int(v['version'])
        latest_id = v['id']
print(latest_id)

bd8fd41f-e449-441f-9ac1-804686a87c55


In [17]:
# the master template contains the vars required, and the body of the template.
templateData = dnac.template_programmer.get_template_details(template_id=templateId)


In [18]:
print(json.dumps(templateData, indent=2))

{
  "name": "interface-des",
  "description": "",
  "tags": [],
  "deviceTypes": [
    {
      "productFamily": "Switches and Hubs"
    }
  ],
  "softwareType": "IOS-XE",
  "softwareVariant": "XE",
  "templateContent": "interface $interface\n  description $description",
  "rollbackTemplateContent": "",
  "templateParams": [
    {
      "parameterName": "interface",
      "dataType": null,
      "defaultValue": null,
      "description": null,
      "required": true,
      "notParam": false,
      "paramArray": false,
      "displayName": null,
      "instructionText": null,
      "group": null,
      "order": 1,
      "selection": null,
      "range": [],
      "key": null,
      "provider": null,
      "binding": "",
      "id": "de633ffd-2072-4fcf-a65b-0edc355a4a50"
    },
    {
      "parameterName": "description",
      "dataType": null,
      "defaultValue": null,
      "description": null,
      "required": true,
      "notParam": false,
      "paramArray": false,
      "displayN

In [19]:
payload = {
    "templateId": latest_id,
    "forcePushTemplate" : True,
    "targetInfo": [
     {

        "id": "10.10.20.81",
        "type": "MANAGED_DEVICE_IP",
        "params": {"description": "my Desc", "interface":"g1/0/12"}
        }
     ]
    }

In [20]:
# this is a hack for the proxy in sandbox.  means we can use RW API with RO creds.  #primativeRBAC
dnac.base_url="https://sandboxdnac2.cisco.com:8080"


In [21]:
deployment= dnac.template_programmer.deploy_template(payload=payload)

In [22]:
print (json.dumps(deployment,indent=2))

{
  "deploymentId": "Deployment of  Template: bd8fd41f-e449-441f-9ac1-804686a87c55.ApplicableTargets: [10.10.20.81]Template Deployemnt Id: 6067ac5e-7db7-4c2f-a98c-1b4d51355bdb",
  "duration": "0 seconds",
  "endTime": "",
  "startTime": ""
}


In [23]:
deploymentId = deployment.deploymentId.split(":")[-1].strip()
print(deploymentId)

6067ac5e-7db7-4c2f-a98c-1b4d51355bdb


In [24]:
status = dnac.template_programmer.get_template_deployment_status(deploymentId)

In [25]:
print(json.dumps(status,indent=2))

{
  "deploymentId": "6067ac5e-7db7-4c2f-a98c-1b4d51355bdb",
  "devices": [
    {
      "detailedStatusMessage": "Provisioning success for template interface-des",
      "deviceId": "6a49c827-9b28-490b-8df0-8b6c3b582d8a",
      "duration": "0 minutes 1 seconds",
      "endTime": "",
      "ipAddress": "6a49c827-9b28-490b-8df0-8b6c3b582d8a",
      "name": "",
      "startTime": "05:10:04 25/08/2019",
      "status": "IN_PROGRESS"
    }
  ],
  "duration": "0 seconds",
  "endTime": "",
  "projectName": "switching",
  "startTime": "",
  "status": "INIT",
  "templateName": "interface-des",
  "templateVersion": "1"
}
